GitHub  
https://github.com/jingyunliang/vrt  
論文  
https://arxiv.org/abs/2201.12288v1  

<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/VRT_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

forms  
https://colab.research.google.com/notebooks/forms.ipynb 

# ランタイムの設定
「ランタイム」→「ランタイムのタイプを変更」→「ハードウェアアクセラレータ」をGPUに変更

# 実行方法
「ランタイム」→「すべてのセルを実行」を選択

# 環境セットアップ

## GitHubからCode Clone

In [ ]:
%cd /content/
!rm -r VRT

# GitHubからVRTのcodeをclone
!git clone https://github.com/JingyunLiang/VRT.git

%cd /content/VRT

# Commits on Oct 5, 2022
!git checkout 94a5f504eb84aedf1314de5389f45f4ba1c2d022

## ライブラリのインストール

In [ ]:
%cd /content/VRT
!pip install -r requirements.txt
!pip install moviepy==0.2.3.5 imageio==2.4.1

# ライブラリのインポート

In [ ]:
import os
import glob
from google.colab import files
import shutil
import cv2
import matplotlib.pyplot as plt
from pathlib import Path

from moviepy.editor import *
from moviepy.video.fx.resize import resize

# 画像または動画のセットアップ
デモでは下記を使用  
超解像、ノイズ除去  
https://ia800502.us.archive.org/4/items/vd_is_for_everybody/vd_is_for_everybody_512kb.mp4  
blur除去  
https://pixabay.com/ja/videos/%E5%B8%82-%E4%BA%BA-%E6%AD%A9%E3%81%8F-%E8%B2%B7%E3%81%84%E7%89%A9-%E9%83%BD%E5%B8%82-999/  

In [ ]:
%cd /content/VRT

# upload images
!rm -r /content/VRT/testsets/*
!rm -r /content/VRT/results/*

os.makedirs('/content/VRT/testsets/uploaded/frames/000', exist_ok=True)
input_folder = '/content/VRT/testsets/uploaded/'
frame_folder = '/content/VRT/testsets/uploaded/frames/000'
frame_path = '/content/VRT/testsets/uploaded/frames/000/frame%08d.png'

uploaded = files.upload()
for filename in uploaded.keys():
  # 動画のリサイズ
  clip = VideoFileClip(filename)
  print('video size width:', clip.w, 'height:', clip.h)
  #@markdown 動画サイズ変更  両方0で変更なし
  resize_w =  0#@param {type:"integer"}
  resize_h =  0#@param {type:"integer"}
  if resize_h == 0 and resize_w == 0:
    pass
  else:
    if resize_w == 0:
      clip = resize(clip, height=resize_h)
    elif resize_h == 0:
      clip = resize(clip, width=resize_w)
    else:
      clip = resize(clip, width=resize_w, height=resize_h)
      
  #@markdown 動画の長さ変更(秒)  end=0は変更なし
  start = 0 #@param {type:"integer"}
  end =  5#@param {type:"integer"}
  if end != 0:
    if start < end and end <= clip.duration:
      clip = clip.subclip(start, end)

  print('move', filename, 'to', input_folder)
  input_path = os.path.join(input_folder, filename)
  clip.write_videofile(input_path)
   
  if not filename.split('.')[-1].lower() in ['png','jpeg','jpg','tiff','bmp','tif',]: # image or video
    print('動画のフレーム画像を抽出 dst:', frame_folder)
    !ffmpeg -i {input_path} -qscale:v 1 -qmin 1 -qmax 1 -vsync 0 -pix_fmt yuv420p {frame_path}
  else:
    !cp {input_path} {frame_folder}

  !rm {filename}

!ls testsets/uploaded

# 超解像

## Testing arguments
--task:  
001_VRT_videosr_bi_REDS_6frames/  
002_VRT_videosr_bi_REDS_16frames/  
003_VRT_videosr_bi_Vimeo_7frames/  
004_VRT_videosr_bd_Vimeo_7frames/  
005_VRT_videodeblurring_DVD/  
006_VRT_videodeblurring_GoPro/  
007_VRT_videodeblurring_REDS/  
008_VRT_videodenoising_DAVIS  

--folder_lq:  
低解像度ビデオのパス。
アップロードしたデータ以外の場合はベンチマークデータセットをダウンロードして使用
  
--folder_gt:  
ground-truthビデオのパス  
アップロードしたビデオには使用不可。 ベンチマークデータセットのみをサポート

--tile:  
時間、高さ、幅のテストサイズ。 
例として[40,128,128]は、ビデオを40x128x128ビデオクリップとして  
テストすることを意味します。
メモリ不足の場合はサイズを下げること(パフォーマンスは低下)
  
--tile_overlap:  
タイルの重なり合うサイズ。 変更不要

In [ ]:
#@markdown uploadかベンチマークか選択
type = "upload" #@param ["upload", "BT001", "BT002", "BT003", "BT004", "BT005", "BT006", "BT007", "BT008"] {allow-input: false}
#@markdown uploadを選択した場合に超解像、blur除去、ノイズ除去か選択
process_type = "blur" #@param ["sr", "blur", "noise"] {allow-input: false}

# アップロードしたビデオで超解像
if type == "upload":
  if process_type == "sr":
    !python main_test_vrt.py \
      --task 001_VRT_videosr_bi_REDS_6frames \
      --folder_lq "/content/VRT/testsets/uploaded/frames/" \
      --tile 6 128 128 --tile_overlap 2 20 20
  if process_type == "blur":
    !python main_test_vrt.py \
      --task 005_VRT_videodeblurring_DVD \
      --folder_lq "/content/VRT/testsets/uploaded/frames/" \
      --folder_gt "/content/VRT/testsets/uploaded/frames/" \
      --tile 12 128 128 --tile_overlap 2 20 20
  if process_type == "noise":
    !python main_test_vrt.py \
      --task 008_VRT_videodenoising_DAVIS \
      --sigma 10 \
      --folder_lq "/content/VRT/testsets/uploaded/frames/" \
      --folder_gt "/content/VRT/testsets/uploaded/frames/" \
      --tile 12 128 128 --tile_overlap 2 20 20


# 以下ベンチマークテスト
# 001, video sr trained on REDS (6 frames), tested on REDS4
elif type == "BT001":
  !python main_test_vrt.py \
    --task 001_VRT_videosr_bi_REDS_6frames \
    --folder_lq testsets/REDS4/sharp_bicubic \
    --folder_gt testsets/REDS4/GT \
    --tile 40 64 64 --tile_overlap 2 20 20
# 002, video sr trained on REDS (16 frames), tested on REDS4
elif type == "BT002":
  !python main_test_vrt.py -\
    -task 002_VRT_videosr_bi_REDS_16frames \
    --folder_lq testsets/REDS4/sharp_bicubic \
    --folder_gt testsets/REDS4/GT \
    --tile 40 64 64 --tile_overlap 2 20 20
# 003, video sr trained on Vimeo (bicubic), tested on Vid4 and Vimeo
elif type == "BT003":
  !python main_test_vrt.py \
    --task 003_VRT_videosr_bi_Vimeo_7frames \
    --folder_lq testsets/Vid4/BIx4 \
    --folder_gt testsets/Vid4/GT \
    --tile 32 64 64 --tile_overlap 2 20 20
  !python main_test_vrt.py \
    --task 003_VRT_videosr_bi_Vimeo_7frames \
    --folder_lq testsets/vimeo90k/vimeo_septuplet_matlabLRx4/sequences \
    --folder_gt testsets/vimeo90k/vimeo_septuplet/sequences \
    --tile 8 0 0 --tile_overlap 0 20 20
# 004, video sr trained on Vimeo (blur-downsampling), tested on Vid4, UDM10 and Vimeo
elif type == "BT004":
  !python main_test_vrt.py \
    --task 004_VRT_videosr_bd_Vimeo_7frames \
    --folder_lq testsets/Vid4/BDx4 \
    --folder_gt testsets/Vid4/GT \
    --tile 32 64 64 --tile_overlap 2 20 20
  !python main_test_vrt.py \
    --task 004_VRT_videosr_bd_Vimeo_7frames \
    --folder_lq testsets/UDM10/BDx4 \
    --folder_gt testsets/UDM10/GT \
    --tile 32 64 64 --tile_overlap 2 20 20
  !python main_test_vrt.py \
    --task 004_VRT_videosr_bd_Vimeo_7frames \
    --folder_lq testsets/vimeo90k/vimeo_septuplet_BDLRx4/sequences \
    --folder_gt testsets/vimeo90k/vimeo_septuplet/sequences \
    --tile 8 0 0 --tile_overlap 0 20 20
# 005, video deblurring trained and tested on DVD
elif type == "BT005":
  !python main_test_vrt.py \
    --task 005_VRT_videodeblurring_DVD \
    --folder_lq testsets/DVD10/test_GT_blurred \
    --folder_gt testsets/DVD10/test_GT \
    --tile 12 128 128 --tile_overlap 2 20 20
# 006, video deblurring trained and tested on GoPro
elif type == "BT006":
  !python main_test_vrt.py \
    --task 006_VRT_videodeblurring_GoPro \
    --folder_lq testsets/GoPro11/test_GT_blurred \
    --folder_gt testsets/GoPro11/test_GT \
    --tile 12 128 128 --tile_overlap 2 20 20
# 007, video deblurring trained on REDS, tested on REDS4
elif type == "BT007":
  !python main_test_vrt.py \
    --task 007_VRT_videodeblurring_REDS \
    --folder_lq testsets/REDS4/blur \
    --folder_gt testsets/REDS4/GT \
    --tile 12 128 128 --tile_overlap 2 20 20
# 008, video denoising trained on DAVIS (noise level 0-50) and tested on Set8 and DAVIS
elif type == "BT008":
  !python main_test_vrt.py \
    --task 008_VRT_videodenoising_DAVIS \
    --sigma 10 --folder_lq testsets/Set8 \
    --folder_gt testsets/Set8 \
    --tile 12 128 128 --tile_overlap 2 20 20
  !python main_test_vrt.py \
    --task 008_VRT_videodenoising_DAVIS \
    --sigma 10  \
    --folder_lq testsets/DAVIS-test \
    --folder_gt testsets/DAVIS-test \
    --tile 12 128 128 --tile_overlap 2 20 20


# Visualization

In [ ]:
def display(img1, name='input'):
  total_figs = 5
  fig = plt.figure(figsize=(total_figs*12, 14))
  ax1 = fig.add_subplot(1, total_figs, 1) 
  plt.title(f'Frame 0 ({name})', fontsize=30)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, total_figs, 2)
  plt.title(f'Frame 1 ({name})', fontsize=30)
  ax2.axis('off')
  ax3 = fig.add_subplot(1, total_figs, 3)
  plt.title(f'Frame 2 ({name})', fontsize=30)
  ax3.axis('off')
  ax4 = fig.add_subplot(1, total_figs, 4)
  plt.title(f'Frame 3 ({name})', fontsize=30)
  ax4.axis('off')
  ax5 = fig.add_subplot(1, total_figs, 5)
  plt.title(f'Frame 4 ({name})', fontsize=30)
  ax5.axis('off')
  ax1.imshow(img1[0])
  ax2.imshow(img1[1])
  ax3.imshow(img1[2])
  ax4.imshow(img1[3])
  ax5.imshow(img1[4])

# Frameごとのビデオを表示
img = []
for path in sorted(Path('testsets').rglob('*.png'))[:10]:
  img.append(cv2.cvtColor(cv2.imread(f'{path}'), cv2.COLOR_BGR2RGB))
display(img, 'input')
img = []
for path in sorted(Path('results').rglob('*.png'))[:10]:
  img.append(cv2.cvtColor(cv2.imread(f'{path}'), cv2.COLOR_BGR2RGB))
display(img, 'VRT output')

# フレーム画像を動画に変換

In [ ]:
result_frames = glob.glob("/content/VRT/results/*/*/*.png")
result_frame_dir = os.path.dirname(result_frames[0])
src_frames = os.path.join(result_frame_dir, "frame%08d.png")
dst_video = "/content/VRT/results/sr_video.mp4"

!ffmpeg -i {src_frames} -qscale:v 1 -qmin 1 -qmax 1 -vsync 0 {dst_video}

clip = VideoFileClip(dst_video)
# 表示用にリサイズ
clip_resize = resize(clip, height=480)
clip_resize.ipython_display()

# ダウンロード

In [ ]:
# 超解像結果をダウンロード
zip_filename = 'VRT_result.zip'
if os.path.exists(zip_filename):
  os.remove(zip_filename)
os.system(f"zip -r -j {zip_filename} /content/VRT/results/*")
files.download(zip_filename)